In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd

In [2]:
# Constants and data
n_buses = 3
n_lines = 2
n_gens = 2
X = [0.1, 0.1]  # Line reactances
gen_limits = [(70, 95), (70, 95)]  # Generation limits (Pmin, Pmax)
cost_coeffs = [(0.01, 2, 0), (0.015, 1.5, 0)]  # Cost coefficients (c, b, a)
min_demand = [40, 30, 70]  # Minimum demand at each bus
max_demand = [60, 50, 80]  # Maximum demand at each bus


In [3]:
# Generate random demand scenarios
np.random.seed(42)  # For reproducibility
demand_scenarios = np.random.randint(min_demand, max_demand, size=(1000, n_buses))

# Prepare DataFrame to store results
columns = ['Demand1', 'Demand2', 'Demand3', 'Gen1', 'Gen2', 'Cost']
df_results = pd.DataFrame(columns=columns)

In [4]:
results_list = []  # List to store temporary results

for Pd in demand_scenarios:
    # Define variables
    theta = cp.Variable(n_buses)
    Pg = cp.Variable(n_gens)

    # Objective function
    objective = cp.Minimize(sum(cost_coeffs[i][0]*Pg[i]**2 + cost_coeffs[i][1]*Pg[i] + cost_coeffs[i][2] for i in range(n_gens)))

    # Power flow expressions
    Pf = [(theta[i] - theta[i + 1]) / X[i] for i in range(n_lines)]

    # Constraints
    constraints = []
    constraints += [Pg[0] - Pd[0] + Pf[0] == 0]  # Bus 1
    constraints += [-Pf[0] + Pf[1] - Pd[1] == 0] # Bus 2
    constraints += [Pg[1] - Pd[2] - Pf[1] == 0]  # Bus 3

    for i in range(n_gens):
        constraints += [Pg[i] >= gen_limits[i][0], Pg[i] <= gen_limits[i][1]]

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    # Collect results and append to the list
    if problem.status == cp.OPTIMAL:
        results_list.append(pd.Series([Pd[0], Pd[1], Pd[2], Pg.value[0], Pg.value[1], problem.value], index=columns))
    else:
        results_list.append(pd.Series([Pd[0], Pd[1], Pd[2], None, None, None], index=columns))

In [5]:
# Concatenate all results into the DataFrame
df_results = pd.concat([df_results, pd.DataFrame(results_list, columns=columns)], ignore_index=True)

# Display a few results to check
print(df_results.head())

   Demand1  Demand2  Demand3  Gen1  Gen2     Cost
0     46.0     49.0     77.0  93.2  78.8  484.604
1     46.0     48.0     76.0  92.0  78.0  476.900
2     50.0     40.0     77.0  90.2  76.8  465.434
3     43.0     37.0     77.0  84.2  72.8  427.994
4     42.0     31.0     77.0  80.0  70.0  402.500


In [6]:
# export dataset to csv

df_results.to_csv('results.csv', index=False)